In [ ]:
# Process memoized/cached content

In [1]:
import os
import tempfile
import newspaper
import pandas as pd

from newspaper import fulltext, Article
from urllib.parse import urlparse

In [2]:
# Get path for 'memoized' (cached) articles
cached_article_path = os.path.join(tempfile.gettempdir(), '.newspaper_scraper', 'memoized')

# List files
os.listdir(cached_article_path)

['cnn.com.txt']

In [3]:
# Load only political articles
content = []
with open(os.path.join(cached_article_path, 'cnn.com.txt'), "r") as f:
    for line in f.readlines():
        if 'politics' in line:
            content.append(line)

In [4]:
# Remove erroneous non-CNN articles
final_content = []
for url in content:
    if 'cnn.com' in urlparse(url).netloc:
        final_content.append(url.strip())

In [5]:
article_objects = []
for url in final_content:
    article = Article(url)
    article_objects.append(article)

In [7]:
def process_cnn_articles(articles):
    metadata = []
    for article in articles:
        # Download and parse articles
        article.download()
        article.parse()
        # Only fetch articles with authors
        if len(article.authors) > 0:
            # Add metadata to dictionary
            parsed_dict = {
                "title": article.title,
                "authors": article.authors,
                "date": article.publish_date,
                "full_text": article.text,
            }
            # Perform NLP to get more metadata
            article.nlp()
            nlp_dict = {
                "summary": article.summary,
                "keywords": article.keywords    
            }
            # Append NLP metadata to main dictionary
            parsed_dict.update(nlp_dict)
            # Append dictionary to list
            metadata.append(parsed_dict)
    print('Fetched metadata for {} CNN articles'.format(len(metadata)))
    return metadata

In [8]:
cnn_metadata = process_cnn_articles(article_objects)

Fetched metadata for 14 CNN articles


In [9]:
# Transform list to dataframe
cnn_df = pd.DataFrame(cnn_metadata)

In [10]:
# Write to PostgreSQL
cnn_df.to_sql('news_politics', engine, if_exists='append')

NameError: name 'engine' is not defined